In [3]:
# Project: "Real‑Time YOLO‑Based Object Detection System with Interactive Gradio Web Interface"
# Members Name:
# Malik Anees Ahmed (BSAI-132)
# Mudassir (BSAI-196)
# Niaz (BSAI-197)
# Abdulla (BSAI-306)
# Faizan (BSAI-307)

# Import Libraries
import numpy as np
from PIL import Image
import gradio as gr
from ultralytics import YOLO

# Load YOLO Model
MODEL_PATH = "yolov8n.pt"
model = YOLO(MODEL_PATH)

# Define Object Detection Function
def detect_objects(image, conf_threshold=0.25, iou_threshold=0.45):
    if image is None:
        return None, "Please upload an image first."

    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)

    results = model.predict(
        source=image,
        conf=conf_threshold,
        iou=iou_threshold,
        verbose=False
    )

    annotated_array = results[0].plot()
    annotated_image = Image.fromarray(annotated_array)

    names = model.names
    counts = {}

    for box in results[0].boxes:
        cls_id = int(box.cls[0].item())
        label = names.get(cls_id, f"class_{cls_id}")
        counts[label] = counts.get(label, 0) + 1

    if counts:
        lines = [f"{label}: {count}" for label, count in counts.items()]
        summary_text = "Detected objects:\n" + "\n".join(lines)
    else:
        summary_text = "No objects detected above the selected confidence threshold."

    return annotated_image, summary_text

# Define Gradio Interface Components
image_input = gr.Image(
    type="numpy",
    label="Upload Image"
)

conf_slider = gr.Slider(
    minimum=0.0,
    maximum=1.0,
    value=0.25,
    step=0.05,
    label="Confidence threshold"
)

iou_slider = gr.Slider(
    minimum=0.0,
    maximum=1.0,
    value=0.45,
    step=0.05,
    label="IoU threshold"
)

image_output = gr.Image(
    type="pil",
    label="Detection Result"
)

text_output = gr.Textbox(
    label="Detection Summary",
    lines=6
)

# Create and Launch Gradio App
demo = gr.Interface(
    fn=detect_objects,
    inputs=[image_input, conf_slider, iou_slider],
    outputs=[image_output, text_output],
    title="YOLO Object Detection App",
    description=(
        "Upload an image and run real-time object detection using a pretrained "
        "YOLO model. Adjust confidence and IoU thresholds to control detections."
    ),
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e83d8ade9a35d3008f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
